## First Step: create viewsheds based on 6 image locations
1. Load Data
    * DEMS
    * GPS csv file
2. Convert csv file to coordinates
3. Convert GPS coordinates (a feature layer) to shp
4. Create individual shps for each image point
    * Extract relevant values for names (names for new shapefiles)
    * Create 6 copies of the GOS shp (conatining all data) and delete all rows expt 1 point (then save shp)
5. Use Viewshed() to calculate viewshed rasters for each image


<i>Please Note this Notebook was completed within an ArcGIS Pro environment with access to Arcpy</i>

In [1]:
# Import system modules
import arcpy
from arcpy import env
from arcpy.sa import *
from pathlib import Path

In [2]:
########################################## Creating Variables for Project ##################################################
# adjust to accurate location of files
GPS_csv_filepath = r"C:\Users\runac\Downloads\Fall_2022\Programming\project_idea\JupNotebooks\GPS_metadata.csv"
DEM_filepath = r"D:\Fall_2022\Programming\Project\Data\30DEM\USGS_1_n49w114_20210607.tif"
output_filepath = "D:\Fall_2022\Programming\Project\Viewshed_MT_clean\Viewshed_MT_clean.gdb"
Num_image_Coords = 6

In [3]:
def Creating_file_names(path, filename_str, length):
    """
    Provide the path where you'd like to save each file, and the naming string used to name resulting files
    Filenmae_str needs to be formatted as "\<filename>"
    Returns a sequential numbered list of filename paths for the provided length (length = number of image coordinates being used to create viewsheds)
    """
    path_ = path
    list_ = []
    for number in range(length): 
        filename = filename_str + str(number + 1)
        list_.append(path_ + filename)
    
    return list_

In [4]:
#################### Connect image.shp point to each of the points on circle perimeter creating a new polyline shapefile #########
##create a def to be called later that extract the coordinates for each image.shp
def image_coords(path_to_image):
    '''
    Given the shapefile path you can extract the points coorndinates. 
    Returns a arcpy Point
    '''
    with arcpy.da.SearchCursor(path_to_image, ['*']) as cursor:
        for row in cursor:
            #assumption, shp is 1 point, thus only returning 1 point value, if shp with many pts provided will return last pt
            cp = arcpy.Point(row[1][0], row[1][1])
    return cp

def creating_polylines(filepath, anchor_point, connection_points): 
    """
    creates lines from each coordinate (image<#>.shp) to the points plotted on the minimum bounding circle 
    created around the viewshed. 
    filepath: insert desired name for each polyline file created with entire filepath (to .gdb)
    anchor_point: filepath to shapefile for each respective image's coordinates
    connection_points: filepath to the points created on the minimum bounding circle
    Returns a new shapefile made of polylines
    """
    # ##creating new file name and shapefile for polylines
    new_fc = filepath
    name = filepath[(len(output_filepath)+1):len(filepath)]
    #matching the spatial refernce of the DEM file
    sr = arcpy.SpatialReference(4269) 
    arcpy.management.CreateFeatureclass(output_filepath, name, 'POLYLINE', spatial_reference = sr)
    #ADDing a column that designates degree (I believe arc designates N as 0 and rotates clockwise)
    arcpy.management.AddField(name, "Degree", "DOUBLE", None, None, None, "Degree", "NULLABLE", "NON_REQUIRED", '')

    percentage = 0
    
    ##Creating a line for each point on perimeter to corresponding image
    #looping through perimeter points
    with arcpy.da.SearchCursor(connection_points, ['*']) as cursor:
        for row in cursor:
            list_points = arcpy.Array([])
            #calulating the degree sequentially clockwise (adjust percentage if GeneratePointsAlongLines() uses differnt percentage)
            degree = (percentage/100)*360
            
            #declaring start (image location) & end (pts on circle) points for each line
            cp1 = image_coords(anchor_point) #pulling the same image point
            cp2 = arcpy.Point(row[1][0], row[1][1])
            list_points.append(cp2)
            list_points.append(cp1)
            
            #assignng spatial reference (should check to make sure its accurate)
            spatial_reference = arcpy.SpatialReference(4326)
            polyline = arcpy.Polyline(list_points, spatial_reference)
           
            #inserting new line to new feature class
            icur = arcpy.da.InsertCursor(new_fc, ["SHAPE@", "Degree"])
            icur.insertRow([polyline, degree])
            del icur
            
            #updating percentage for each line
            percentage += 0.2


### Edit : declare a spatial reference as a variable (4326?? WGS_1984)
* remove long text inside XYTableToPoint()

In [16]:
########################################## Loading DEM and GPS data ###########################################################
#(steps 1-4)
##relevant rasters needed for the viewshed...
DEM1 = Raster(DEM_filepath)

#convert the csv table to points (run once)
arcpy.management.XYTableToPoint(GPS_csv_filepath, 
                                r"GPS_metadata_XYTableToPoint", 
                                "decimal_Long", 
                                "decimal_Lat",
                                "GPSAltitude", 
                                'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],VERTCS["WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PARAMETER["Vertical_Shift",0.0],PARAMETER["Direction",1.0],UNIT["Meter",1.0]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision')
###converting GPS data to a shp
arcpy.conversion.FeatureClassToShapefile("GPS_metadata_XYTableToPoint", output_filepath)

##creating a list that stores the name of each image file (used later to create individual shp for each point)
List_shp = []
List_names = []
with arcpy.da.UpdateCursor("GPS_metadata_XYTableToPoint", ['Field1']) as cursor:
    for row in cursor:
        print(row[0])
        name = row[0].split('.')
        List_shp.append(f'{name[0]}.shp')
        List_names.append(row[0])
        
##copying featureclass and converting to individual shpfiles for each point
data = output_filepath + "\GPS_metadata_XYTableToPoint"
count = 0
for name in List_shp:
    #copying original file 
    arcpy.management.CopyFeatures(data, name)
    #creating shpfiles with 1 point by deleting all features excepts for 1 row
    with arcpy.da.UpdateCursor(name, ['Field1','decimal_La','decimal_Lo', 'GPSAltit_1']) as cursor:
        for row in cursor:
            if row[0] != List_names[count]:
                cursor.deleteRow()
    count += 1

image1.HEIC
image2.HEIC
image3.HEIC
image4.JPG
image5.JPG
image8.JPG


In [17]:
############### Creating a list of names to use for viewsheds, based on the number of images processed ###################
# (step 5)
###declaring lists for names and creating files for Viewshed Outputs
List_Raster_V_names = Creating_file_names(output_filepath, "\outvwdshd", Num_image_Coords)
List_outnames = Creating_file_names(output_filepath, "\outViewshed", Num_image_Coords)

###################################################### VIEWSHED ############################################################
for shp, out_file, vwshd in zip(List_shp, List_outnames, List_Raster_V_names):
    # Execute Viewshed (DON'T NEED 'List_Raster_V_names')
    outViewshed = Viewshed(DEM1, shp, 1, "FLAT_EARTH", 0.13)
    # Save the output 
    outViewshed.save(out_file)
    name = vwshd[(len(output_filepath)+1):len(vwshd)]
    print("saving", name)


saving outvwdshd1
saving outvwdshd2
saving outvwdshd3
saving outvwdshd4
saving outvwdshd5
saving outvwdshd6


## Next big step: extracting elevation values (maxes) in a 360 view around image points. 

1. Viewshed raster to Polygon
2. Create a minimum bounding circle on vectorized viewshed raster
3. Generate points on perimeter of circle (min bounding region)
4. Connect image.shp point to each of the points on circle perimeter creating a new polyline shapefile.
5. Use vectorized viewshed raster to mask DEM (resulting in a viewshed shaped DEM with real elevation values)
6. Extract the cell values from the clipped viewshed DEM with the Polylines. (HERE CURRENTLY!!)
    * For each line store the largest DEM value and save values in a table
7. Plot table (in LATIS see Visualization_elev_dist.ipynb notebook)

In [18]:
################################################## Viewshed raster to Polygon ########################################
##vectorizing resulting viewshed rasters

#list of names of the viewshed tifs
vwshd_tifs = Creating_file_names(output_filepath, "\outViewshed", Num_image_Coords)
#list of names to be used for vectorized viewsheds
ras_to_polys = Creating_file_names(output_filepath, "\Poly_outView", Num_image_Coords)

###### Looping over "ras_to_polys" to convert ALL viewsheds to polygons
for tif, poly in zip(vwshd_tifs, ras_to_polys):
    #isolating names of tif file from rest of path
    name = tif[(len(output_filepath)+ 1):len(tif)]
    #converting rasters to polygons
    arcpy.conversion.RasterToPolygon(name, 
                                     poly, 
                                     "SIMPLIFY", 
                                     "Value", 
                                     "SINGLE_OUTER_PART", 
                                     None)

# #deleting polygons with gridcode = 0 (represents areas viewer cannot see)
for v in ras_to_polys: 
    with arcpy.da.UpdateCursor(v, ['*']) as cursor:
        for row in cursor:
            if row[3] == 0:
                cursor.deleteRow()
    print("Saving & Updating:", v[(len(output_filepath)+1):len(v)])

Saving & Updating: D:\Fall_2022\Programming\Project\Viewshed_MT_clean\Viewshed_MT_clean.gdb\Poly_outView1
Saving & Updating: D:\Fall_2022\Programming\Project\Viewshed_MT_clean\Viewshed_MT_clean.gdb\Poly_outView2
Saving & Updating: D:\Fall_2022\Programming\Project\Viewshed_MT_clean\Viewshed_MT_clean.gdb\Poly_outView3
Saving & Updating: D:\Fall_2022\Programming\Project\Viewshed_MT_clean\Viewshed_MT_clean.gdb\Poly_outView4
Saving & Updating: D:\Fall_2022\Programming\Project\Viewshed_MT_clean\Viewshed_MT_clean.gdb\Poly_outView5
Saving & Updating: D:\Fall_2022\Programming\Project\Viewshed_MT_clean\Viewshed_MT_clean.gdb\Poly_outView6


In [19]:
########################### create a min bounding circle on vectorized viewshed raster #######################
# creating list of names for minimum bounding circles
min_bound_names = Creating_file_names(output_filepath, "\B_outView", Num_image_Coords)

# creating minimum bounding circle around viewshds
for poly_vwshd, min_bound in zip(ras_to_polys, min_bound_names):
    arcpy.management.MinimumBoundingGeometry(poly_vwshd, 
                                         min_bound, 
                                         "CIRCLE", 
                                         "ALL")
    print("Saving", min_bound[(len(output_filepath)+1):len(min_bound)])

Saving \B_outView1
Saving \B_outView2
Saving \B_outView3
Saving \B_outView4
Saving \B_outView5
Saving \B_outView6


In [20]:
############################## Generate points on perimeter of circle (min bounding region) ######################
# creating list of names for points on minimum bouding circles
pts_names = Creating_file_names(output_filepath, "\Pts_View", Num_image_Coords)

# creating pts on minimum bounding circle
for circle, pts in zip(min_bound_names, pts_names):
    arcpy.management.GeneratePointsAlongLines(circle, 
                                          pts, 
                                          "PERCENTAGE",  
                                          Percentage=0.2, 
                                          Include_End_Points='END_POINTS')
    
    print("Saving", pts[(len(output_filepath)+1):len(pts)])

    ##takes apporx 2-3 mins to execute

Saving Pts_View1
Saving Pts_View2
Saving Pts_View3
Saving Pts_View4
Saving Pts_View5
Saving Pts_View6


In [25]:
#################### Connect image.shp point to each of the points on circle perimeter creating a new polyline shapefile ##########

# creating paths and names for new polyline files 
polyline_names = Creating_file_names(output_filepath, "\View_lines", Num_image_Coords)

# polyline_names : recently declared above
# pts_names : calling this already made list of minimum bounding circle pts
# List_shp : calling this already made list of image locations 

#Running the function that'll create polylines for each image and corresponding viewshed points
for polyline, image, pts, in zip(polyline_names, List_shp, pts_names):
    print("Creating:", polyline[(len(output_filepath) + 1):len(polyline)])
    creating_polylines(polyline, image, pts)
    
#takes 1.5 mins to make lines when p = 0.2

Creating: View_lines1
Creating: View_lines2
Creating: View_lines3
Creating: View_lines4
Creating: View_lines5
Creating: View_lines6


In [28]:
########################################## Use vectorized viewshed raster to mask DEM ###################################
# creating paths and names for new clipped DEM of viewsheds 
c_DEM_names = Creating_file_names(output_filepath, "\Clipped_DEM_Vwshd", Num_image_Coords)

#declaring the spatial extent of the DEM1 (used to help clip the DEM to the viewshed output)
extent = DEM1.extent
xmin = extent.XMin
ymin = extent.YMin
xmax = extent.XMax
ymax = extent.YMax
spatial_extent = str(xmin) + " " + str(ymin) +  " " + str(xmax) +  " " +str(ymax)

#creating a clipped extent of DEM for each image
for vwshd, clip_dem in zip(ras_to_polys, c_DEM_names):
    print("Clipping:", clip_dem[(len(output_filepath)+1):len(clip_dem)])
    poly_vwshd = vwshd[(len(output_filepath)+1):len(vwshd)]
    arcpy.management.Clip(DEM1, 
                          spatial_extent,
                          clip_dem, 
                          poly_vwshd, 
                          "-999999", 
                          "ClippingGeometry", 
                          "NO_MAINTAIN_EXTENT")

Clipping: Clipped_DEM_Vwshd1
Clipping: Clipped_DEM_Vwshd2
Clipping: Clipped_DEM_Vwshd3
Clipping: Clipped_DEM_Vwshd4
Clipping: Clipped_DEM_Vwshd5
Clipping: Clipped_DEM_Vwshd6


In [32]:
########################################## Extract the Elevation values from the clipped viewshed DEM with the Polylines ############
# creating paths and names for new INTIGER-(datatype) clipped DEM of viewsheds 
int_DEM_names = Creating_file_names(output_filepath, "\Int_clip_DEM_Vwshd", Num_image_Coords)
poly_pixels_vw = Creating_file_names(output_filepath, "\Poly_Int_Clip_DEM", Num_image_Coords)
max_elevation = Creating_file_names(output_filepath, "\lines_max_elev", Num_image_Coords)

##convert elevation datatype from decimal to integer (looses some data)
for clip_dem, int_dem in zip(c_DEM_names, int_DEM_names):
    print("Saving", int_dem[(len(output_filepath)+1):len(int_dem)])
    out_raster = arcpy.ia.Int(clip_dem[(len(output_filepath)+1):len(clip_dem)]); out_raster.save(int_dem)

##convert raster to polygons (s.t. each raster pixel is a polygon)
for int_dem, poly_dem in zip(int_DEM_names, poly_pixels_vw):
    arcpy.conversion.RasterToPolygon(int_dem[(len(output_filepath)+1):len(int_dem)], 
                                     poly_dem, 
                                     "NO_SIMPLIFY", 
                                     "Value", 
                                     "SINGLE_OUTER_PART", 
                                     None)

##spatial join between pixels (as polygons) and polylines : whilst saving only max elevations for each polyline
##this join returns the max 'gridcode' ('elevation value') that intersects with a line!!
##need to figure out how to apply 'field mapping' for each vectorized-pixel viewshed..?
for lines, poly_dem, elev in zip(polyline_names, poly_pixels_vw, max_elevation):
    arcpy.analysis.SpatialJoin(lines[(len(output_filepath)+1):len(lines)], 
                               poly_dem[(len(output_filepath)+1):len(poly_dem)], 
                               elev, 
                               "JOIN_ONE_TO_ONE", 
                               "KEEP_ALL", 
                               'Degree "Degree" true true false 19 Double 0 0,First,#,view1_lines_b,Degree,-1,-1;gridcode "gridcode" true true false 4 Long 0 0,Max,#,Poly_Int_Clip_DEM1,gridcode,-1,-1', 
                               "INTERSECT", 
                               None, 
                               '')
    print("Saving Max Elevation value")

Saving Int_clip_DEM_Vwshd1
Saving Int_clip_DEM_Vwshd2
Saving Int_clip_DEM_Vwshd3
Saving Int_clip_DEM_Vwshd4
Saving Int_clip_DEM_Vwshd5
Saving Int_clip_DEM_Vwshd6
Saving Max Elevation value
Saving Max Elevation value
Saving Max Elevation value
Saving Max Elevation value
Saving Max Elevation value
Saving Max Elevation value


In [42]:
##exported the shp as a csv to view in LATIS
out_path = r"D:\Fall_2022\Programming\Project\Viewshed_MT_clean"
csv_names = Creating_file_names(out_path, "\lines_max_elev", Num_image_Coords)

for elev, csv in zip(max_elevation, csv_names):
    name = csv + ".csv"
    print("Saving:", name[(len(out_path)+1):(len(csv)+4)])
    arcpy.conversion.ExportTable(elev[(len(output_filepath)+1):len(elev)], 
                                 name, 
                                 '', 
                                 "NOT_USE_ALIAS", 
                                 'Join_Count "Join_Count" true true false 4 Long 0 0,First,#,view1_lines_b_SpatialJoin2,Join_Count,-1,-1;TARGET_FID "TARGET_FID" true true false 4 Long 0 0,First,#,view1_lines_b_SpatialJoin2,TARGET_FID,-1,-1;Degree "Degree" true true false 8 Double 0 0,First,#,view1_lines_b_SpatialJoin2,Degree,-1,-1;gridcode "gridcode" true true false 4 Long 0 0,First,#,view1_lines_b_SpatialJoin2,gridcode,-1,-1', 
                                 None)

Saving: lines_max_elev1.cs
Saving: lines_max_elev2.cs
Saving: lines_max_elev3.cs
Saving: lines_max_elev4.cs
Saving: lines_max_elev5.cs
Saving: lines_max_elev6.cs


##### After successfully extrating the Max elevation values for each polyline, I need to measure distance from observer/image pt to location of Max elevation pixel. I then need to create new values where max elevation is divided by distance (to accurately depict a mountain ridgeline w.r.t. distance) 



### Another Big Step: Calculate Distance from image/anchor pt to each Max Elevation 

1. Spatial Join between vectorized clipped DEM from viewshed and the polylines generated from image to min bounding pts
    * retain polygon (pixel) geometry
    * make sure it retains only MAX elevation polygon per line
2. Clip or divide the Polylines with the geom returned from previous step. 
    * retain only lines that intersect with image/anchor point
3. Measure the line length (Use "Calculate Geometry Attributes" tool)

### <i> Only running following code on 1 image.shp file, need to later apply to all </i> 

In [42]:
poly_pixels_vw = Creating_file_names(output_filepath, "\Poly_Int_Clip_DEM", Num_image_Coords)

poly_pixels_vw[0]

'D:\\Fall_2022\\Programming\\Project\\Viewshed_MT_clean\\Viewshed_MT_clean.gdb\\Poly_Int_Clip_DEM1'

In [43]:
############################## Spatial Join between lines and DEM pixels Keeping Elevation Geom ###############################
#creating a Spatial Join where we keep the elevation geom: 
arcpy.analysis.SpatialJoin(poly_pixels_vw[0], 
                           "lines_max_elev1", 
                           r"D:\Fall_2022\Programming\Project\Viewshed_MT_clean\Viewshed_MT_clean.gdb\Poly_Int_Clip_DEM_SpJn1", 
                           "JOIN_ONE_TO_MANY", 
                           "KEEP_COMMON", 
                           'gridcode "gridcode" true true false 4 Long 0 0,First,#,Poly_Int_Clip_DEM1,gridcode,-1,-1;Degree "Degree" true true false 19 Double 0 0,First,#,view1_lines_b,Degree,-1,-1;Line_ID "Line_ID" true true false 19 Double 0 0,First,#,view1_lines_b,Line_ID,-1,-1', 
                           "INTERSECT", 
                           None, 
                           '')

<Result 'D:\\Fall_2022\\Programming\\Project\\Viewshed_MT_clean\\Viewshed_MT_clean.gdb\\Poly_Int_Clip_DEM_SpJn1'>

In [44]:
############################# ONly woeking way to indentify MAX for each polyline... #######################################
## identifying the max elevation for each polyline and saving the polygon OBJECTID, coords, and elevation to a dictionary

max_dict = {}
counter = 0
#For each line:
with arcpy.da.SearchCursor("lines_max_elev1", ['Degree']) as cursor:
    for line in cursor:
        ID = line[0] #make sure to grab "Degree" so as to match unique values later
    #iterate over corresponding pixels that match each line (through 'ID' column)
        with arcpy.da.SearchCursor("Poly_Int_Clip_DEM_SpJn1", ['*']) as cur:
            Max_ = 0
            obj_id = 0
            coords = 0
            for pixel in cur:
                counter += 1
                #make sure these indeces match correct columns!!
                if pixel[6] == ID:
                    if Max_ < pixel[5]:
                        Max_ = pixel[5]
                        obj_id = pixel[3] #target_ID
                        coords = pixel[1]
                    max_dict[ID] = [obj_id, coords, Max_]
#         print("Max:", Max_, "for Polygon:", obj_id )   

print("Finally Done!", counter)
### Fucntion works (with print functions) BUT I bet there is a better/faster way...

Finally Done! 7881231


In [45]:
# a simply check: delete me later
print(len(max_dict)) #length should be equal to number of polylines (in this case 501)
from pprint import pprint
pprint(max_dict)

501
{0.0: [506, (-113.5173611108113, 48.735416666975226), 2465],
 0.72: [542, (-113.51597222192234, 48.735138889197486), 2454],
 1.44: [625, (-113.51430555525567, 48.734583333641865), 2451],
 2.16: [723, (-113.51277777747777, 48.734027778086244), 2449],
 2.88: [769, (-113.51124999969997, 48.733750000308504), 2461],
 3.6: [824, (-113.50972222192217, 48.733472222530764), 2477],
 4.32: [629, (-113.50819444414431, 48.734583333641865), 2444],
 5.039999999999999: [482, (-113.50680555525541, 48.735694444752966), 2401],
 5.76: [429, (-113.50527777747756, 48.73625000030859), 2423],
 6.4799999999999995: [401, (-113.50374999969976, 48.736527778086355), 2401],
 7.199999999999999: [374, (-113.50236111081077, 48.73680555586421), 2374],
 7.92: [120, (-113.5004166663663, 48.74138888919768), 2388],
 8.64: [126, (-113.49874999969956, 48.74125000030875), 2408],
 9.360000000000001: [131, (-113.49736111081069, 48.74125000030875), 2414],
 10.080000000000002: [97, (-113.49560185155127, 48.7425462966051), 240

 78.47999999999983: [8978, (-113.51208333303333, 48.575138889192175), 2297],
 79.19999999999983: [8978, (-113.51208333303333, 48.575138889192175), 2297],
 79.91999999999983: [8978, (-113.51208333303333, 48.575138889192175), 2297],
 80.63999999999983: [8978, (-113.51208333303333, 48.575138889192175), 2297],
 81.35999999999983: [8075, (-113.50097222192188, 48.58097222252567), 2622],
 82.07999999999983: [8076, (-113.50069444414414, 48.58097222252567), 2629],
 82.79999999999983: [8014, (-113.49986111081071, 48.58125000030344), 2657],
 83.51999999999981: [8015, (-113.49958333303292, 48.58125000030344), 2663],
 84.23999999999981: [7956, (-113.49847222192177, 48.58152777808121), 2690],
 84.95999999999981: [7629, (-113.49430555525501, 48.583472222525785), 2816],
 85.67999999999981: [7752, (-113.49541666636611, 48.582638889192395), 2784],
 86.3999999999998: [7669, (-113.4934722219216, 48.58319444474793), 2805],
 87.1199999999998: [7669, (-113.4934722219216, 48.58319444474793), 2805],
 87.839999

 154.80000000000027: [11118, (-113.50180555525525, 48.5581944447471), 2388],
 155.52000000000027: [11117, (-113.50208333303299, 48.5581944447471), 2406],
 156.24000000000026: [11115, (-113.50263888858855, 48.5581944447471), 2422],
 156.9600000000003: [11052, (-113.50319444414416, 48.558472222524955), 2426],
 157.6800000000003: [11050, (-113.50374999969976, 48.558472222524955), 2444],
 158.4000000000003: [10982, (-113.50430555525534, 48.558750000302695), 2450],
 159.12000000000032: [10981, (-113.5045833330331, 48.558750000302695), 2455],
 159.84000000000032: [10979, (-113.50513888858868, 48.558750000302695), 2469],
 160.56000000000034: [10978, (-113.50541666636644, 48.558750000302695), 2473],
 161.28000000000034: [10977, (-113.50569444414421, 48.558750000302695), 2480],
 162.00000000000037: [10909, (-113.50624999969979, 48.559027778080434), 2490],
 162.72000000000037: [10908, (-113.5065277774776, 48.559027778080434), 2495],
 163.4400000000004: [10906, (-113.5070833330332, 48.55902777808

 230.4000000000013: [9277, (-113.51736111081134, 48.57291666696983), 2431],
 231.1200000000013: [9277, (-113.51736111081134, 48.57291666696983), 2431],
 231.84000000000137: [9276, (-113.51763888858912, 48.57291666696983), 2419],
 232.56000000000137: [9275, (-113.51791666636686, 48.57291666696983), 2411],
 233.28000000000137: [9254, (-113.51819444414465, 48.5731944447476), 2381],
 234.00000000000136: [9253, (-113.51861111081135, 48.5731944447476), 2387],
 234.72000000000142: [9253, (-113.51861111081135, 48.5731944447476), 2387],
 235.44000000000142: [9253, (-113.51861111081135, 48.5731944447476), 2387],
 236.16000000000142: [9253, (-113.51861111081135, 48.5731944447476), 2387],
 236.88000000000142: [9253, (-113.51861111081135, 48.5731944447476), 2387],
 237.60000000000144: [9253, (-113.51861111081135, 48.5731944447476), 2387],
 238.32000000000144: [9252, (-113.519027777478, 48.5731944447476), 2385],
 239.04000000000144: [9250, (-113.51958333303358, 48.5731944447476), 2380],
 239.7600000

 307.44000000000244: [4458, (-113.57458333303542, 48.64708333363896), 2554],
 308.1600000000024: [4321, (-113.5740277774799, 48.64763888919455), 2572],
 308.88000000000244: [4139, (-113.57305555525761, 48.64819444475009), 2589],
 309.60000000000247: [1036, (-113.63847222192643, 48.73180555586404), 2982],
 310.3200000000025: [3666, (-113.57208333303534, 48.649305555861275), 2610],
 311.04000000000246: [3268, (-113.57152777747974, 48.650138889194665), 2616],
 311.7600000000025: [3003, (-113.57069444414645, 48.65111111141684), 2623],
 312.4800000000025: [2885, (-113.5699999997019, 48.65152777808353), 2638],
 313.2000000000025: [2840, (-113.56930555525744, 48.65180555586139), 2637],
 313.9200000000025: [2807, (-113.5684722219241, 48.652083333639126), 2610],
 314.64000000000254: [2848, (-113.56708333303519, 48.65180555586139), 2579],
 315.3600000000025: [2851, (-113.56597222192399, 48.65180555586139), 2564],
 316.08000000000254: [2853, (-113.56513888859067, 48.65180555586139), 2568],
 316.8

In [46]:
#grabbing the OBJECTID's and using these values to delete non-max associated polygons
Obj_ID_list = []
for item in max_dict.values():
    #there are pixel polygons IDs that are duplicated in this list, only need to add them once (no need for repeating IDs)
    if item[0] not in Obj_ID_list:
        Obj_ID_list.append(item[0])

safety = 0
#deleteting polygons whose target_IDs are NOT the max_dict max_dict) 
with arcpy.da.UpdateCursor("Poly_Int_Clip_DEM_SpJn1", ['*']) as cursor:
    for row in cursor:
        if row[3] not in Obj_ID_list and safety < 4:
#             print(row[3])
#             safety +=1
            cursor.deleteRow()
            
#still need to delete more pixel polygons if there do not conatin the max elevetion pre line...(slower than the above Search Cursor)
with arcpy.da.UpdateCursor("Poly_Int_Clip_DEM_SpJn1", ['*']) as cursor:
    for row in cursor:
        #checking if the Target Id and the elevation are within the max_dict, if not delete the row
        if row[3] not in max_dict[row[6]] and row[5] not in max_dict[row[6]]:
            cursor.deleteRow()

In [64]:
######################################### Clip line where they intersect MAX pixels ##########################################

#rename "Target_FID" col inside thevectorixed pixel geom fetures that have been spatialy joined with lines...
# make a new field then copy the old field values into the new one 
arcpy.management.CalculateField("Poly_int_Clip_DEM_SpJn1", "PIXEL_ID", "!TARGET_FID!", "PYTHON3", '', "LONG", "NO_ENFORCE_DOMAINS")
#delete unnecessary columns
arcpy.management.DeleteField("Poly_int_Clip_DEM_SpJn1", "TARGET_FID", "DELETE_FIELDS")

#creating points that delinate the intersect of each line with each max polygon (creates points whenver each line intersect a polygon)
arcpy.analysis.Intersect(r"'lines with MAX elev\lines_max_elev1' #;'Measure distances for Viewshed 1\Poly_Int_Clip_DEM_SpJn1' #", 
                         r"D:\Fall_2022\Programming\Project\Viewshed_MT_clean\Viewshed_MT_clean.gdb\lines_poly_Intersect1", 
                         "NO_FID", 
                         None, 
                         "POINT")

#need to keep only one point per line that marks where each line intersects withits max elevation pixel/polygon
safety = 0
number_of_matches = 0
with arcpy.da.UpdateCursor("lines_poly_Intersect1", ['gridcode', 'Degree', 'PIXEL_ID']) as cursor:
    for row in cursor:
        safety += 1
        if safety < 10000:
#             print(row)
            degree = row[1]
            #if pixel_id are equal and elevation values are equal keep
            if  max_dict[degree][0] == row[2] and max_dict[degree][2] == row[0]:
#                 print("found a match")
                number_of_matches += 1
            # else delete row
            else:
#                 print("deleting ", degree)
                cursor.deleteRow()

print("Number of matches found:", number_of_matches)      


## Split lines by points:
arcpy.management.SplitLineAtPoint("lines_max_elev1", 
                                  "lines_poly_Intersect1", 
                                  "lines_Split_view1", 
                                  None)

## delete polyline with ORG_SEG = 1 (delete lines that do not intersect with anchor point):
safety = 0
with arcpy.da.UpdateCursor("lines_Split_view1", ['ORIG_SEQ']) as cursor:
    for row in cursor:
        if row[0] == 1:
            cursor.deleteRow()
            
## Successfully clipped lines!!!


In [65]:
########################################### Calculate Geometry (line Length)################################
# Calculating Line Length for each line (geodesic) into a new column named "LENGTH"
arcpy.management.CalculateGeometryAttributes("lines_Split_view1", 
                                             "LENGTH LENGTH_GEODESIC", 
                                             "KILOMETERS", '', 
                                             'GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', 
                                             "SAME_AS_INPUT")

<Result 'lines_Split_view1'>

In [ ]:
#exporting Distances associated with each of the polylines...
arcpy.conversion.ExportTable("lines_Split_view1", 
                             r"D:\Fall_2022\Programming\Project\Viewshed_MT_clean\lines_Split_view1.csv", 
                             '', 
                             "NOT_USE_ALIAS", 
                             'Join_Count "Join_Count" true true false 4 Long 0 0,First,#,lines_Split_view1,Join_Count,-1,-1;TARGET_FID "TARGET_FID" true true false 4 Long 0 0,First,#,lines_Split_view1,TARGET_FID,-1,-1;Degree "Degree" true true false 8 Double 0 0,First,#,lines_Split_view1,Degree,-1,-1;gridcode "gridcode" true true false 4 Long 0 0,First,#,lines_Split_view1,gridcode,-1,-1;ORIG_FID "ORIG_FID" true true false 4 Long 0 0,First,#,lines_Split_view1,ORIG_FID,-1,-1;ORIG_SEQ "ORIG_SEQ" true true false 4 Long 0 0,First,#,lines_Split_view1,ORIG_SEQ,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,lines_Split_view1,Shape_Length,-1,-1;LENGTH "LENGTH" true true false 8 Double 0 0,First,#,lines_Split_view1,LENGTH,-1,-1', 
                             None)

### Scratch Work